### kafka console consumer to validate pushed events
kafka-console-consumer.bat --topic pizzario_customer_offers_python --bootstrap-server localhost:9092

In [5]:
import pandas as pd
import json
segment_and_offers = pd.read_json("Segements_and_offers.json", lines=True)
customers_data = pd.read_csv("cluster_results.csv")

In [6]:
def find_offer(customerId):
    cluster = list(customers_data[customers_data["CustomerID"] == customerId]['cluster'])[0]
    offer = list(segment_and_offers[segment_and_offers["segment"] == cluster]['offer'])[0]
    return offer

In [23]:
from kafka import KafkaProducer  
from json import dumps
def send_to_kafka(message):
    my_producer = KafkaProducer(
        bootstrap_servers = ['localhost:9092'],
        value_serializer = lambda x:dumps(x).encode('utf-8')
    )
    my_producer.send('pizzario_customer_offers_python', value=message)
    print("sent to kafka topic pizzario_customer_offers_python")

In [24]:
from json import loads  
from kafka import KafkaConsumer 
import json
from datetime import datetime
my_consumer = KafkaConsumer(  
    'pizzario_customer_visit',  
     bootstrap_servers = ['localhost : 9092'],  
     #auto_offset_reset = 'earliest',  
     enable_auto_commit = True,  
     #group_id = 'my-group',  
     value_deserializer = lambda x : loads(x.decode('utf-8'))  
     )  

Pizzario_outlets_coordinates = [('O1', 10, 15), ('O2', 25, 51) ]

for kmessage in my_consumer:  
    #message = json.loads(message.value)
    message = kmessage.value
    
    time_of_arrival = datetime.utcfromtimestamp(kmessage.timestamp/1000).strftime('%Y-%m-%d %H:%M:%S')
    
    nearest_outlets = []
    for o in Pizzario_outlets_coordinates:
        distance_from_pizzario = abs(o[1] - message['coordinates'][0]) + abs(o[2] - message['coordinates'][1])
        if distance_from_pizzario < 30:
            nearest_outlets.append((o[0],distance_from_pizzario))
        offer = ""
        if len(nearest_outlets) > 0:
            customer_id = message['customerid']
            offer = find_offer(customer_id)
    message['time_of_arrival'] = time_of_arrival
    message['nearest_outlets'] = nearest_outlets
    message['offer'] = offer
    send_to_kafka(message)
    print(message)

sent to kafka topic pizzario_customer_offers_python
{'customerid': 66, 'coordinates': [41, 18], 'time_of_arrival': '2022-02-12 17:08:52', 'nearest_outlets': [], 'offer': ''}
sent to kafka topic pizzario_customer_offers_python
{'customerid': 67, 'coordinates': [36, 10], 'time_of_arrival': '2022-02-12 17:08:57', 'nearest_outlets': [], 'offer': ''}
sent to kafka topic pizzario_customer_offers_python
{'customerid': 68, 'coordinates': [37, 40], 'time_of_arrival': '2022-02-12 17:09:02', 'nearest_outlets': [('O2', 23)], 'offer': 'O2'}
sent to kafka topic pizzario_customer_offers_python
{'customerid': 69, 'coordinates': [31, 14], 'time_of_arrival': '2022-02-12 17:09:07', 'nearest_outlets': [('O1', 22)], 'offer': 'O2'}
sent to kafka topic pizzario_customer_offers_python
{'customerid': 70, 'coordinates': [35, 20], 'time_of_arrival': '2022-02-12 17:09:12', 'nearest_outlets': [], 'offer': ''}
sent to kafka topic pizzario_customer_offers_python
{'customerid': 71, 'coordinates': [16, 18], 'time_of_a

KeyboardInterrupt: 